# Choosing the best location for the oil well

The goal is to find the best place for a new oil well of the mining company X.
We have data on oil samples from 3 regions. The data set for each region has 100,000 oil wells with measured oil quality and volume of reserves. We need to build a machine learning model that will help to pick the region with the highest profit margin. We'll analyze potential profits and risks using the Bootstrap technique.

# Data description
- id - unique oil well identifier
- f0, f1, f2 - three features of points (their specific meaning is unimportant, but the features themselves are significant)
- product - volume of reserves in the oil well (thousand barrels). This's our target.

# Study the general information

In [84]:
# importing libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
%config InlineBackend.figure_format = 'retina'

In [85]:
# opening the data
df_geo_0 = pd.read_csv('geo_data_0.csv')
df_geo_1 = pd.read_csv('geo_data_1.csv')
df_geo_2 = pd.read_csv('geo_data_2.csv')

In [104]:
# printing 3 rows from each of datasets
datasets = {'0':df_geo_0,'1':df_geo_1,'2':df_geo_2}
for key, val in datasets.items():
    print('Region',key,':')
    display(val.head(3)) 
    print()

Region 0 :


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647



Region 1 :


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305



Region 2 :


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910


In [112]:
# printing general information
for key, val in datasets.items():
    print('Region',key,':')
    display(val.info())
    print()

Region 0 :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None


Region 1 :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None


Region 2 :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

In [106]:
# checking numeric values
for key, val in datasets.items():
    print('Region',key,':')
    display(val.describe()) 
    print()

Region 0 :


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347



Region 1 :


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408



Region 2 :


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


In [113]:
# checking for duplicates in 'id' column
for key, val in datasets.items():
    print('df_geo', key, 'duplicates:', val.id.duplicated().sum())
    print()

df_geo 0 duplicates: 10

df_geo 1 duplicates: 4

df_geo 2 duplicates: 4



### Conclusion

- We have 3 dataframes with 100000 rows.
- We have duplicates in "id" column. Just drop them.
- "f0", "f1", "f2" are our features. "product" is our target.
- So we can drop "id" column.

### Step 2. Data preprocessing

*Drop duplicates*

In [62]:
# drop duplicates in "id" column
for df in (df_geo_0, df_geo_1, df_geo_2):
    df.drop_duplicates(subset='id', inplace = True)

*Drop "id" column*

In [63]:
# drop duplicates in "id" column
for df in (df_geo_0, df_geo_1, df_geo_2):
    df.drop(['id'], axis=1, inplace = True)

*Check the values*

In [64]:
for df in (df_geo_0, df_geo_1, df_geo_2):
    print(df.shape[0])

99990
99996
99996


### Step 3. Feature Preparation and training the model

*We need to train and test the model for each region*

In [65]:
# choose our datasets
df_geo = (df_geo_0, df_geo_1, df_geo_2)

# create lists for targets and pridictions
targets = []
predictions = []

# create dataframe for our metrics
metrics = pd.DataFrame()

for i in range(len(df_geo)):
    # create features and target datasets
    features = df_geo[i].drop(['product'], axis=1)
    target = df_geo[i]['product']

    # split the data into training and validation sets
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, test_size=0.25, random_state=12345)

    # create the model
    model = LinearRegression()

    # fit the model
    model.fit(features_train, target_train)

    # predict values
    predicted_valid = model.predict(features_valid)

    # find values for the constant model, it predicts the mean target value for each observation
    predicted_valid_const = pd.Series(target_train.mean(), index=target_valid.index)

    # calculate rmse, rmse_const
    rmse = mean_squared_error(target_valid, predicted_valid) ** 0.5
    rmse_const = mean_squared_error(target_valid, predicted_valid_const) ** 0.5

    # calculate volume of reserves for now and in the future
    vol_valid = target_valid.mean()
    vol_predicted = predicted_valid.mean()
    
    # append target_valid, predicted_valid to lists targets and predictions
    targets.append(list(target_valid))
    predictions.append(list(predicted_valid))

    # append rmse, rmse_const, vol_valid, vol_valid to metrics dataframe
    metrics = metrics.append({'rmse': rmse,
                              'rmse_const': rmse_const,
                              'vol_predicted': vol_predicted,
                              'vol_valid': vol_valid}, ignore_index=True)

*Look at our metrics for different regions*

In [66]:
# use index to rename rows in metrics dataframe
metrics.index = ('df_geo_0', 'df_geo_1', 'df_geo_2')
metrics.round(2)

,rmse,rmse_const,vol_predicted,vol_valid
df_geo_0,37.85,44.38,92.79,92.16
df_geo_1,0.89,45.90,69.18,69.19
df_geo_2,40.08,44.68,94.87,94.79


### Conclusion

- So, we have the best predicted volume of reserves for "df_geo_2" = 94.87, but RMSE = 40.08.  
- And we have best RMSE = 0.89 for "df_geo_1", but Volume = 69.18 is lowest.
- Our conclusion isn't clear,  we cannot choose the best region yet. Also, we cannot trust models with such scores. So we need to go deeper and bootstrap method will help with that.

### Step 4. Prepare for profit calculation

Calculate the volume of reserves sufficient for developing a new well without losses:
- number of wells for Bootstrap = 200 wells
- the revenue from one unit of product (volume of reserves is in thousand barrels) = 4500 USD

*Calculate how much will be cost 1 well*

In [67]:
cost_per_unit = int(100000000 / 200) # one well development cost
revenue_per_unit = 4500 # the revenue from 1 barrel

*Calculate how many barrels do we need to produce*

In [68]:
vol_not_los = round(cost_per_unit / revenue_per_unit, 2)
vol_not_los

111.11

### Conclusion

- So we need to produce the minimum 111.12 barrels for one well. And we won't have losses.
- If we compare the obtained value with the average volume of reserves in each region, we can establish that maximum was 95.0. This means that we definitely have unprofitable wells in each region. Hence we choose 200 wells with best predictions.

### Step 5. Write a function to calculate profit 

*Write a function to calculate profit from a set of 200 selected oil wells and model predictions*

In [69]:
def profit(target, predictions, count, revenue_unit, cost_unit):
    # sort our target predictions
    predictions_sorted = pd.Series(predictions).sort_values(ascending=False)
    
    # take indices from predictions_sorted and use it with for target
    selected_values = pd.Series(target)[predictions_sorted.index][:count]
    
    # calculate total product volume
    product_vol = selected_values.sum()
    
    # calculate revenue
    revenue = product_vol * revenue_unit
    
    # calculate cost
    cost = cost_unit * count
    
    # return gross profit 
    return (revenue - cost).round(2)

*Calculate profit for "df_geo_0", "df_geo_1", "df_geo_2"*

In [70]:
for i in range(len(df_geo)):
    print(profit(targets[i], predictions[i], 200, 4500, 500000))

33651872.38
24150866.97
25012838.53


### Conclusion

We have best gross profit in the 1st region = 33.65 million dollars. So we let's check it with Bootstrap.

### Step 6. Use the bootstrap technique

*Use the bootstrap technique with 1000 iterations and 500 samples to find the distribution of profit*

In [71]:
def bootstrap_profit(target_bootstrap, predicted_bootstrap):
    # create a RandomState() instance from the numpy.random module
    state = np.random.RandomState(12345)
    
    # create a list of metrics values
    values = []
    params = pd.DataFrame()
    
    for i in range(1000):
        # make 500 samples
        target_subsample = pd.Series(target_bootstrap).sample(n=500, replace=True, random_state=state)
        
        # take indices from target_subsample and use it with for predicted_valid_subsample
        predicted_valid_subsample = pd.Series(predicted_bootstrap)[target_subsample.index]
        
        # append values from profit function to list "values"
        values.append(profit(target_subsample, predicted_valid_subsample, 200, 4500, 500000))

    # convert values to Series
    values = pd.Series(values)
    
    # calculate mean, confidence interval limits and risk of losses for "values" 
    average_profit = round(values.mean(), 2)
    lower_limit = values.quantile(0.025).round(2)
    upper_limit = values.quantile(0.975).round(2)
    risk_of_losses = (((values < 0).sum() / len(values)) * 100).round(2)
    
    print('average_profit:', average_profit)
    print('confidence_interval:', lower_limit, ':', upper_limit )
    print('risk_of_losses:', risk_of_losses)

*Calculate profit for "df_geo_0", "df_geo_1", "df_geo_2" with Bootstrap technique*

In [72]:
for i in range(len(df_geo)):
    print('Region',i,':')
    bootstrap_profit(targets[i], predictions[i])
    print()

Region 0 :
average_profit: 4094280.39
confidence_interval: -1315360.29 : 9443955.82
risk_of_losses: 7.1

Region 1 :
average_profit: 5369025.88
confidence_interval: 1145279.03 : 9707879.11
risk_of_losses: 0.9

Region 2 :
average_profit: 3537903.63
confidence_interval: -1591940.31 : 8756525.03
risk_of_losses: 10.2



### Conclusion

- As we see we have negative gross profit (for confidence interval) and percent of losses is 7.1 for the 1st region. Hence the 1st region isn't good for us. 
- We have best average profit in the 2nd region. This region has highest average profit = 5.37 million dollars. And the lowest risk of losses = 0.9

### Step 7. Overall conclusion

*The main task:*

Build a model that will help to pick the region with the highest profit margin. Analyze potential profit and risks using the Bootstrap technique.

*Conclusion:*

Bootstrap has improved our results. And we've chosen the 2nd region.